<a href="https://colab.research.google.com/github/chetan-957/Data-Management-sp2025/blob/main/ps0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **1️⃣ Load & Inspect Datasets**

In [14]:
import gdown
import pandas as pd

# Google Drive file ID
file_id = "1QB05L0xO9XE6l-U5NQNrAkWsUPxOZpF_"

# Direct download URL
download_url = f"https://drive.google.com/uc?id={file_id}"

# Define output file name
output_file = "FoodBalanceSheets_E_All_Data.csv"

# Download the file
gdown.download(download_url, output_file, quiet=False)

# Load CSV
faostat_df = pd.read_csv(output_file, encoding="latin1")

# Display sample data
print("FAOSTAT Data Sample:\n", faostat_df.head())


Downloading...
From (original): https://drive.google.com/uc?id=1QB05L0xO9XE6l-U5NQNrAkWsUPxOZpF_
From (redirected): https://drive.google.com/uc?id=1QB05L0xO9XE6l-U5NQNrAkWsUPxOZpF_&confirm=t&uuid=b07dbf87-7141-4a21-88e4-8a3bf65ccacc
To: /content/FoodBalanceSheets_E_All_Data.csv
100%|██████████| 119M/119M [00:00<00:00, 159MB/s]


FAOSTAT Data Sample:
    Area Code Area Code (M49)         Area  Item Code Item Code (FBS)  \
0          2            '004  Afghanistan       2501          'S2501   
1          2            '004  Afghanistan       2901          'S2901   
2          2            '004  Afghanistan       2901          'S2901   
3          2            '004  Afghanistan       2901          'S2901   
4          2            '004  Afghanistan       2901          'S2901   

          Item  Element Code                                 Element  \
0   Population           511           Total Population - Both sexes   
1  Grand Total           664           Food supply (kcal/capita/day)   
2  Grand Total           661                      Food supply (kcal)   
3  Grand Total           674  Protein supply quantity (g/capita/day)   
4  Grand Total           671             Protein supply quantity (t)   

           Unit        Y2010  ... Y2019N        Y2020  Y2020F Y2020N  \
0       1000 No     28189.67  ...    NaN

In [15]:
import pandas as pd

# GitHub Raw File Link
github_url = "https://raw.githubusercontent.com/chetan-957/Data-Management-sp2025/main/P_Data_Extract_From_World_Development_Indicators.xlsx"

# Load WDI Excel file directly from GitHub
wdi_df = pd.read_excel(github_url, sheet_name="Data")

# Display sample data
print("WDI Data Sample:\n", wdi_df.head())


WDI Data Sample:
   Country Name Country Code  \
0  Afghanistan          AFG   
1  Afghanistan          AFG   
2  Afghanistan          AFG   
3  Afghanistan          AFG   
4  Afghanistan          AFG   

                                         Series Name     Series Code  \
0                       GDP per capita (current US$)  NY.GDP.PCAP.CD   
1         GNI per capita, Atlas method (current US$)  NY.GNP.PCAP.CD   
2                                         Gini index     SI.POV.GINI   
3  Poverty headcount ratio at national poverty li...     SI.POV.NAHC   
4  Poverty headcount ratio at $2.15 a day (2017 P...     SI.POV.DDAY   

  1980 [YR1980] 1981 [YR1981] 1982 [YR1982] 1983 [YR1983] 1984 [YR1984]  \
0            ..            ..            ..            ..            ..   
1            ..            ..            ..            ..            ..   
2            ..            ..            ..            ..            ..   
3            ..            ..            ..            ..     

### **2️⃣ Extract Relevant Columns**

In [16]:
# Select only relevant elements from FAOSTAT
faostat_selected_elements = [
    "Food supply (kcal/capita/day)", "Food supply quantity (kg/capita/yr)",
    "Protein supply quantity (g/capita/day)", "Fat supply quantity (g/capita/day)",
    "Production (tonnes)", "Import Quantity (tonnes)", "Export Quantity (tonnes)",
    "Stock Variation (tonnes)", "Domestic supply quantity (tonnes)", "Losses (tonnes)"
]

# Filter FAOSTAT dataset
faostat_filtered = faostat_df[faostat_df["Element"].isin(faostat_selected_elements)]
faostat_filtered = faostat_filtered[["Area", "Item", "Element"] + [col for col in faostat_df.columns if col.startswith("Y")]]
faostat_filtered = faostat_filtered.rename(columns={"Area": "Country", "Item": "Food Category", "Element": "Metric"})

# Extract relevant indicators from WDI
wdi_selected_indicators = ["NY.GDP.PCAP.CD", "NY.GNP.PCAP.CD", "SI.POV.GINI", "NE.TRD.GNFS.ZS"]

wdi_filtered = wdi_df[wdi_df["Series Code"].isin(wdi_selected_indicators)]
wdi_filtered = wdi_filtered.rename(columns={"Country Name": "Country", "Series Name": "Indicator"})

# Clean year columns (remove " [YRxxxx]" format)
wdi_filtered.columns = [col.split(" [YR")[0] if " [YR" in col else col for col in wdi_filtered.columns]


### **3️⃣ Handle Missing Data**

In [17]:
# Drop metadata columns (e.g., "Y2010F", "Y2010N") from FAOSTAT
faostat_filtered = faostat_filtered[[col for col in faostat_filtered.columns if not col.endswith(("F", "N"))]]

# Convert year columns to numeric in FAOSTAT
year_cols_fao = [col for col in faostat_filtered.columns if col.startswith("Y")]
faostat_filtered[year_cols_fao] = faostat_filtered[year_cols_fao].apply(pd.to_numeric, errors='coerce')

# Replace ".." in WDI dataset with NaN and convert year columns to numeric
wdi_filtered.replace("..", pd.NA, inplace=True)
year_cols_wdi = [col for col in wdi_filtered.columns if col.isdigit()]
wdi_filtered[year_cols_wdi] = wdi_filtered[year_cols_wdi].apply(pd.to_numeric, errors='coerce')

# Check missing data summary
print("FAOSTAT Missing Data Summary:\n", faostat_filtered.isnull().sum())
print("WDI Missing Data Summary:\n", wdi_filtered.isnull().sum())


FAOSTAT Missing Data Summary:
 Country             0
Food Category       0
Metric              0
Y2010            8589
Y2011            8651
Y2012            8090
Y2013            8058
Y2014            6383
Y2015            6357
Y2016            6334
Y2017            6046
Y2018            6160
Y2019            2526
Y2020            2034
Y2021            1901
Y2022            1911
dtype: int64
WDI Missing Data Summary:
 Country           0
Country Code      0
Indicator         0
Series Code       0
1980            437
1981            430
1982            426
1983            422
1984            416
1985            408
1986            402
1987            389
1988            394
1989            382
1990            343
1991            340
1992            306
1993            311
1994            305
1995            278
1996            279
1997            280
1998            266
1999            272
2000            254
2001            265
2002            236
2003            233
2004            2

<ipython-input-17-ea4de75a252a>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  faostat_filtered[year_cols_fao] = faostat_filtered[year_cols_fao].apply(pd.to_numeric, errors='coerce')


### **4️⃣ Standardize Country Names**

In [18]:
# Dictionary for FAOSTAT to WDI country name mapping
country_name_mapping = {
    "Bolivia (Plurinational State of)": "Bolivia", "Iran (Islamic Republic of)": "Iran",
    "United Republic of Tanzania": "Tanzania", "Venezuela (Bolivarian Republic of)": "Venezuela",
    "Republic of Korea": "Korea, Rep.", "Democratic People's Republic of Korea": "Korea, Dem. People's Rep.",
    "United States of America": "United States", "TÃ¼rkiye": "Turkey",
    "Democratic Republic of the Congo": "Congo, Dem. Rep.", "Congo": "Congo, Rep.",
    "CÃ´te d'Ivoire": "Côte d'Ivoire", "Iran": "Iran, Islamic Rep.", "Turkey": "Turkiye",
    "Venezuela": "Venezuela, RB"
}

# Apply the mapping
faostat_filtered["Country"] = faostat_filtered["Country"].replace(country_name_mapping)

# Remove non-country regions from FAOSTAT dataset
non_countries = {"Africa", "Asia", "Europe", "World", "Land Locked Developing Countries", "Small Island Developing States"}
faostat_filtered = faostat_filtered[~faostat_filtered["Country"].isin(non_countries)]


### **5️⃣ Merge FAOSTAT & WDI Data**

In [20]:
# Identify valid year columns (only numeric values)
year_cols_wdi = [col for col in wdi_filtered.columns if col.isdigit()]

# Keep only relevant columns in WDI (Country, Indicator, Series Code, and Year Columns)
wdi_filtered = wdi_filtered[["Country", "Indicator", "Series Code"] + year_cols_wdi]

# Convert "Year" column names to integers
wdi_long = wdi_filtered.melt(id_vars=["Country", "Indicator", "Series Code"],
                              var_name="Year", value_name="Value")

# ✅ Remove any remaining non-numeric values in the Year column
wdi_long = wdi_long[wdi_long["Year"].str.isnumeric()]

# Convert "Year" column to integer type
wdi_long["Year"] = wdi_long["Year"].astype(int)

# Merge datasets on Country & Year
merged_data = faostat_long.merge(wdi_long, on=["Country", "Year"], how="inner")

# Save merged dataset as CSV
merged_data.to_csv("Merged_FAOSTAT_WDI.csv", index=False)

# Display merged dataset sample
print("Merged Dataset Sample:\n", merged_data.head())


Merged Dataset Sample:
        Country Food Category                                  Metric  Year  \
0  Afghanistan   Grand Total           Food supply (kcal/capita/day)  2010   
1  Afghanistan   Grand Total           Food supply (kcal/capita/day)  2010   
2  Afghanistan   Grand Total           Food supply (kcal/capita/day)  2010   
3  Afghanistan   Grand Total           Food supply (kcal/capita/day)  2010   
4  Afghanistan   Grand Total  Protein supply quantity (g/capita/day)  2010   

   Value_x                                   Indicator     Series Code  \
0  2208.00                GDP per capita (current US$)  NY.GDP.PCAP.CD   
1  2208.00  GNI per capita, Atlas method (current US$)  NY.GNP.PCAP.CD   
2  2208.00                                  Gini index     SI.POV.GINI   
3  2208.00                            Trade (% of GDP)  NE.TRD.GNFS.ZS   
4    65.76                GDP per capita (current US$)  NY.GDP.PCAP.CD   

      Value_y  
0  560.621505  
1  530.000000  
2         NaN 